<h1> Recuperação de Informação </h1>
<h2> Trabalho 1 - Coletor </h2>
<h2>Leonardo Resende </h2>
<img style="float:left" src="imgs/vamos_coletar.png"><img style="float:left" src="imgs/me.png" width=255>

---

<h3> Parte 1: A Coleta</h3><br>

O coletor é estruturado dentro do conceito teórico estudado em aula:

   * Temos uma estrutura que representa os `domínios`, assim como suas restrições de acesso
   <br>
   * Temos uma estrutura que representa o `escalonador`, organizando as páginas coletadas e repassando as próximas para o `baixador`
   <br>
   * Temos uma estrutura que representa o `baixador`, visitando as páginas e coletando novas url para coleta

In [1]:
from crawler.scheduler import Scheduler
from crawler.page_fetcher import PageFetcher
from urllib.parse import urlparse

In [2]:
usr_agent = 'citycrawler (ricitycrawler.wordpress.com)'
page_limit = 1
depth_limit = 1

In [3]:
obj_url = [urlparse('https://prefeitura.pbh.gov.br')]
obj_scheduler = Scheduler(usr_agent, page_limit, depth_limit, obj_url)
obj_fetcher = PageFetcher(obj_scheduler)

URL https://prefeitura.pbh.gov.br inserida!


In [4]:
obj_fetcher.run()

In [5]:
# Páginas coletadas
obj_scheduler.page_count

1

In [6]:
# URLs coletadas
len(obj_scheduler.set_discovered_urls)

17

Vemos acima um exemplo de uma coleta!<br>
Transformamos a string em um objeto url, que é convertida para um `domínio` (Domain) dentro do `escalonador` (Scheduler), vemos o mesmo com um user_agent (identificação do coletor), o limite e profundidade da coleta.<br>
Por último, vemos que o `baixador` (PageFetcher) recebe o `escalonador` e roda com uma página.<br>

<h4>Repare que visitamos UMA página, mas dentro dela coletamos muitas outras urls.<br>
Isso é importante para distinguir que coletar uma PÁGINA é diferente de coletar uma URL </h4>

---

<h3> Parte 2: Políticas & Polidez</h3><br>

Voltando ao `user_agent` mencionado previamente:

   * Páginas web tem, em sua maioria, um arquivo chamado `robots.txt` na raiz do domínio
   <br>
   * Esse arquivo indica o que a administração do domínio permite ou não deseja que seja coletado
   <br>
   * É boa prática que coletores se identifiquem

Estes pontos são importantes e para isso, usamos um método que baixa e implementa as regras do `robots.txt` de cada domínio que visitamos!

In [7]:
from crawler.scheduler import Scheduler
from urllib.parse import urlparse
from urllib import robotparser

In [8]:
usr_agent = 'citycrawler (ricitycrawler.wordpress.com)'
obj_scheduler = Scheduler(usr_agent, 1, 1, [])

In [9]:
obj_url = urlparse('https://prefeitura.pbh.gov.br')
'''Podemos visitar??''', obj_scheduler.can_fetch_page(obj_url)

('Podemos visitar??', True)

In [10]:
obj_url = urlparse('https://prefeitura.pbh.gov.br/web.config')
'''Podemos visitar??''', obj_scheduler.can_fetch_page(obj_url)

('Podemos visitar??', False)

Por último, foi criado um site no wordpress para que a administração de um site que fique curiosa com os acessos ao domínio possa obter mais informações:
<br><br>
<a style="font-family:Papyrus; font-size:2em;" href="https://ricitycrawler.wordpress.com" title="Crawler HomePage">RI City Crawler</a>
<img style="float:center" src="imgs/site.jpg">

<h3> Parte 3: Mãos na massa</h3><br>

Segue a coleta, utilizando várias `threads` que se encontra no arquivo `coleta.txt`:

In [11]:
from crawler.scheduler import Scheduler
from crawler.page_fetcher import PageFetcher
from urllib.parse import urlparse

In [12]:
usr_agent = 'citycrawler (ricitycrawler.wordpress.com)'
page_limit = 5000
depth_limit = 6
thread_number = 200
url_seeds = ['https://prefeitura.pbh.gov.br',
             'https://www.capital.sp.gov.br',
             'https://prefeitura.rio',
             'https://prefeitura.poa.br',
             'https://www.goiania.go.gov.br',
             'https://www.fortaleza.ce.gov.br',
             'https://www.curitiba.pr.gov.br',
             'http://www.belem.pa.gov.br',
             'https://www.df.gov.br',
             'https://www.saopaulo.sp.leg.br',
             'https://www.pmf.sc.gov.br',
             'https://www.natal.rn.gov.br',
             'https://boavista.rr.gov.br',
             'https://macae.rj.gov.br',
             'https://www.vitoria.es.gov.br/',
             'https://www.ibge.gov.br'
            ]
arr_url_seeds = []
for url in url_seeds:
    arr_url_seeds.append(urlparse(url))

In [ ]:
obj_scheduler = Scheduler(usr_agent, page_limit, depth_limit, arr_url_seeds)

arr_obj_fetcher = []
for _ in range(thread_number):
    arr_obj_fetcher.append(PageFetcher(obj_scheduler))
    arr_obj_fetcher[-1].start()
for thread in arr_obj_fetcher:
    thread.join()

URL https://prefeitura.pbh.gov.br inserida!
URL https://www.capital.sp.gov.br inserida!
URL https://prefeitura.rio inserida!
URL https://prefeitura.poa.br inserida!
URL https://www.goiania.go.gov.br inserida!
URL https://www.fortaleza.ce.gov.br inserida!
URL https://www.curitiba.pr.gov.br inserida!
URL http://www.belem.pa.gov.br inserida!
URL https://www.df.gov.br inserida!
URL https://www.saopaulo.sp.leg.br inserida!
URL https://www.pmf.sc.gov.br inserida!
URL https://www.natal.rn.gov.br inserida!
URL https://boavista.rr.gov.br inserida!
URL https://macae.rj.gov.br inserida!
URL https://www.vitoria.es.gov.br/ inserida!
URL https://www.ibge.gov.br inserida!
The page count has reached a new milestone: 100
The page count has reached a new milestone: 200
The page count has reached a new milestone: 300
The page count has reached a new milestone: 400
The page count has reached a new milestone: 500
The page count has reached a new milestone: 600
The page count has reached a new milestone: 70

In [ ]:
obj_scheduler.page_count

<h3> Parte 4: Mar de Threads</h3><br>

Usando uma coleta de 100 páginas, foram obtidos os seguintes dados de threads x tempo levado:

   * Os pontos da análise das threads foram: 1, 6, 11, 16, 21, 30, 50, 70, 90, 110, 130, 150<br>
   <br>
   * O pior resultado foi com uma thread, quase 110 segundos para a coleta<br>
   <br>
   * O melhor resultado foi com 150 threads com o tempo de 42 segundos (talvez essa seja a pergunta para a resposta??)<br>
   <br>
   * O interessante é que com 70 threads, tivemos uma performance semelhante, mais testes serão feitos pra além do relatório.

<img style="float:center" src="imgs/myplot.png">

A análize foi feita no arquivo "~/analisis/main.py" e editada com procreate para visualização personalizada